In [1]:
from mpa.models import Mpa, mpa_post_save
from django.db.models.signals import post_save, post_delete
from mpa.views import mpas_all_nogeom
import reversion
from reversion.models import Revision
from reversion.models import Version
import datetime
from django.db.models import Q, F, Func, Value
from django.db.models.functions import Concat

from importlib import reload

In [2]:
# Update mpatlas field values before data import
mpas = mpas_all_nogeom.exclude(
        Q(country__icontains='MEX') | Q(country__icontains='CHL') | Q(country__icontains='IDN') | 
        Q(country__icontains='CHN') | Q(country__icontains='JPN') | 
        Q(sovereign__icontains='MEX') | Q(sovereign__icontains='CHL') | Q(sovereign__icontains='IDN') |
        Q(sovereign__icontains='CHN') | Q(sovereign__icontains='JPN')
    ).filter(wdpa_id__isnull=False)
mpas.count()

20659

In [17]:
mpas.update(
    no_take_mpatlas=F('no_take'),
    no_take_area_mpatlas=F('no_take_area'),
    calc_area_mpatlas=F('calc_area'),
    calc_m_area_mpatlas=F('calc_m_area'),
    status_mpatlas=F('status'),
    status_year_mpatlas=F('status_year'),
)

16082

In [18]:
# Set wdpa_pid for new zones
# If existing MPAtlas sites with this wdpa_id,
# set first non-rejected site to the first PID_A.
# Clear pid for all other MPAtlas sites and flag
# as orphaned zones.

wzones = []
mpasw = [float(wi) for wi in mpas_all_nogeom.filter(wdpa_id__isnull=False).order_by('wdpa_id').values_list('wdpa_id', flat=True)]
polyset = Wdpa2019Poly.objects.filter(wdpaid__in=mpasw).only('wdpaid', 'wdpa_pid', 'name', 'desig_eng', 'iso3', 'parent_iso3')
for w in polyset:
    if w.wdpa_pid != str(int(w.wdpaid)):
        wzones.append(w.wdpaid)
wzones = list(set(wzones))
len(wzones)

433

In [19]:
# Assign records with zones to be the first zone.  Do not rerun.
count = 0
for wid in wzones:
    zones = Wdpa2019Poly.objects.filter(wdpaid=wid).only('wdpaid', 'wdpa_pid', 'name', 'desig_eng', 'iso3', 'parent_iso3')
    firstpid = zones.order_by('wdpa_pid').first().wdpa_pid
    mpaz = mpas_all_nogeom.filter(wdpa_id=int(wid))
    for m in mpaz:
        print(m.mpa_id, m.wdpa_id, m.wdpa_pid)
        m.wdpa_pid = firstpid
        count += 1
        print(count, m.mpa_id, m.wdpa_id, m.wdpa_pid)
        #m.save()


1389 18432 18432_A
1 1389 18432 18432_A
14978 555556866 555556866_A
2 14978 555556866 555556866_A
68808839 555556866 555556866_B
3 68808839 555556866 555556866_A
68808840 555556866 555556866_C
4 68808840 555556866 555556866_A
14980 555556868 555556868_A
5 14980 555556868 555556868_A
68808841 555556868 555556868_B
6 68808841 555556868 555556868_A
3874 305157 305157_A
7 3874 305157 305157_A
14982 555556870 555556870_A
8 14982 555556870 555556870_A
68808842 555556870 555556870_B
9 68808842 555556870 555556870_A
14983 555556871 555556871_A
10 14983 555556871 555556871_A
68808843 555556871 555556871_B
11 68808843 555556871 555556871_A
68808844 555556874 555556874_B
12 68808844 555556874 555556874_A
14986 555556874 555556874_A
13 14986 555556874 555556874_A
68808845 555556874 555556874_C
14 68808845 555556874 555556874_A
9898 126987 126987_A
15 9898 126987 126987_A
68808749 126987 126987_B
16 68808749 126987 126987_A
68808846 555556875 555556875_B
17 68808846 555556875 555556875_A
14987 5555

5726 555516382 555516382_A
147 5726 555516382 555516382_A
5727 555516383 555516383_A
148 5727 555516383 555516383_A
5729 555516385 555516385_A
149 5729 555516385 555516385_A
5730 555516386 555516386_A
150 5730 555516386 555516386_A
5731 555516387 555516387_A
151 5731 555516387 555516387_A
5732 555516388 555516388_A
152 5732 555516388 555516388_A
5733 555516389 555516389_A
153 5733 555516389 555516389_A
5734 555516390 555516390_A
154 5734 555516390 555516390_A
5735 555516391 555516391_A
155 5735 555516391 555516391_A
5736 555516392 555516392_A
156 5736 555516392 555516392_A
5737 555516393 555516393_A
157 5737 555516393 555516393_A
5738 555516394 555516394_A
158 5738 555516394 555516394_A
5739 555516395 555516395_A
159 5739 555516395 555516395_A
5740 555516396 555516396_A
160 5740 555516396 555516396_A
5741 555516397 555516397_A
161 5741 555516397 555516397_A
5742 555516398 555516398_A
162 5742 555516398 555516398_A
5743 555516399 555516399_A
163 5743 555516399 555516399_A
5744 555516400

68808801 555548087 555548087_B
314 68808801 555548087 555548087_A
14583 555548087 555548087_A
315 14583 555548087 555548087_A
68808802 555548087 555548087_C
316 68808802 555548087 555548087_A
3320 167356 167356_A
317 3320 167356 167356_A
4334 308676 308676_A
318 4334 308676 308676_A
7704334 308676 308676_A
319 7704334 308676 308676_A
68808762 308676 308676_B
320 68808762 308676 308676_A
68808763 308676 308676_C
321 68808763 308676 308676_A
68808803 555548101 555548101_B
322 68808803 555548101 555548101_A
14588 555548101 555548101_A
323 14588 555548101 555548101_A
68808805 555548106 555548106_C
324 68808805 555548106 555548106_A
14589 555548106 555548106_A
325 14589 555548106 555548106_A
68808804 555548106 555548106_B
326 68808804 555548106 555548106_A
68808807 555548108 555548108_C
327 68808807 555548108 555548108_A
68808806 555548108 555548108_B
328 68808806 555548108 555548108_A
14590 555548108 555548108_A
329 14590 555548108 555548108_A
68808764 308684 308684_B
330 68808764 308684 3

12457 555515590 555515590_A
466 12457 555515590 555515590_A
5670 555515592 555515592_A
467 5670 555515592 555515592_A
5671 555515593 555515593_A
468 5671 555515593 555515593_A
5672 555515594 555515594_A
469 5672 555515594 555515594_A
1356 18133 18133_A
470 1356 18133 18133_A
1357 18136 18136_A
471 1357 18136 18136_A
1358 18137 18137_A
472 1358 18137 18137_A
1359 18138 18138_A
473 1359 18138 18138_A
5673 555515611 555515611_A
474 5673 555515611 555515611_A
1360 18144 18144_A
475 1360 18144 18144_A
5674 555515625 555515625_A
476 5674 555515625 555515625_A
1362 18161 18161_A
477 1362 18161 18161_A
1363 18164 18164_A
478 1363 18164 18164_A
1364 18166 18166_A
479 1364 18166 18166_A
2070 65277 65277_A
480 2070 65277 65277_A
1365 18178 18178_A
481 1365 18178 18178_A
2071 65294 65294_A
482 2071 65294 65294_A
12458 555515666 555515666_A
483 12458 555515666 555515666_A
9812 67347 67347_A
484 9812 67347 67347_A
9751 18197 18197_A
485 9751 18197 18197_A
2135 67351 67351_A
486 2135 67351 67351_A
21

In [4]:
# If we set more than one site to the same PID, then keep the pid only for the first non-rejected site.
# Set the others to be flagged as orphaned zones.

dups = (
    mpas.filter(wdpa_id__isnull=False, wdpa_pid__contains='_')
    .values('wdpa_pid')
    .annotate(pidcount=Count('wdpa_pid'))
    .values('wdpa_pid')
    .order_by()
    .filter(pidcount__gt=1)
)

duppids = mpas.filter(wdpa_id__isnull=False).filter(wdpa_pid__in=dups)

for pid in dups.values_list('wdpa_pid', flat=True):
    dupmpas = mpas.filter(wdpa_pid=pid).order_by('mpa_id')
    nonrejects = dupmpas.exclude(verification_state='Rejected as MPA').order_by('mpa_id')
    if nonrejects:
        m = nonrejects[0]
    else:
        m = dupmpas[0]
    others = dupmpas.exclude(mpa_id=m.mpa_id)
    for o in others:
        o.categories.add('Orphaned Zone')
    others.update(wdpa_pid=None, notes=Concat(F('notes'), Value(' ORPHANED ZONE, please clean up.')))

In [ ]:
from django.db.models import F, Func
mpazz = mpas_all_nogeom.filter(wdpa_id__isnull=False).exclude(wdpa_pid__in=wd)
print(mpazz.count())
mpazz.update(wdpa_pid=Func(F('wdpa_id'), function='CAST', template='%(function)s(%(expressions)s as varchar(52))'))

In [ ]:
mpazx = mpas_all_nogeom.filter(wdpa_id__isnull=False).exclude(wdpa_id__in=[int(w) for w in wzones])
mpazx.update(wdpa_pid=Func(F('wdpa_id'), function='CAST', template='%(function)s(%(expressions)s as varchar(52))'))

In [ ]:
count = 0
mpazy = mpas_all_nogeom.only('wdpa_id', 'mpa_id', 'wdpa_pid').filter(wdpa_id__isnull=False).exclude(wdpa_id__in=[int(w) for w in wzones])
for m in mpazy:
    count += 1
    mywid = m.wdpa_id
    mypid = str(m.wdpa_id)
    myid = m.mpa_id
    m.wdpa_pid = mypid
    print(count, m.mpa_id, m.wdpa_id, m.wdpa_pid)
    #m.save()
    mq = mpas_all_nogeom.filter(mpa_id=myid).update(wdpa_pid=mypid)